In [268]:
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import re

data = pd.read_csv('train_test.csv')
data.head()

,Target,Description,Description1,Description2,Description3,Description4
0,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...,NaN,NaN,deputato/deputata\nministro/ministra\nsenatore...,deputata del Parlamento europeo\nmembro del Pa...
1,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...,NaN,NaN,presidente di regione,presidente di giunta regionale\ngovernatore di...
2,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...,NaN,NaN,NaN,NaN
3,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...,NaN,NaN,assessore comunale/assessora comunale\nsindaco...,membro del consiglio comunale\nconsigliera com...
4,"Ambasciatori, ministri plenipotenziari ed alti...","Ambasciatore\nAmbasciatori, ministri plenipote...",NaN,NaN,ambasciatore/ambasciatrice\nconsigliere di amb...,addetto diplomatico\naddetta diplomatica\ncons...


In [269]:
columns = data.columns.tolist()
columns

['Target',
 'Description',
 'Description1',
 'Description2',
 'Description3',
 'Description4']

In [270]:
for i in columns:
    data[i] = data[i].apply(lambda x: str(x))

In [271]:
data['descrizione_completa'] = data['Description'] + data['Description1'] + data['Description2'] + \
                                data['Description3'] + data['Description4']

In [272]:
data = data.drop(columns[1:],axis = 1)
data.head()

,Target,descrizione_completa
0,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...
1,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...
2,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...
3,Membri di organismi di governo e di assemblee ...,Membri di organismi di governo e di assemblee ...
4,"Ambasciatori, ministri plenipotenziari ed alti...","Ambasciatore\nAmbasciatori, ministri plenipote..."


In [273]:
data.descrizione_completa = data.descrizione_completa.apply(lambda x: re.split('; |, |\*|\n|/',x))
data.descrizione_completa = data.descrizione_completa.apply(lambda x: ' '.join(x))
data.descrizione_completa.iloc[0]

'Membri di organismi di governo e di assemblee nazionali con potestà legislativa e regolamentare Membro del consiglio dei ministri Membro del senato Membro della camera dei deputatinannandeputato deputata ministro ministra senatore senatricedeputata del Parlamento europeo membro del Parlamento europeo membro del parlamento deputato deputata deputato del Parlamento europeo membro della camera dei deputati ministra del governo ministro ministra ministro del governo senatrice senatore'

In [274]:
data.descrizione_completa = data.descrizione_completa.apply(lambda x: word_tokenize(x))
data.descrizione_completa.iloc[0][:10]

['Membri',
 'di',
 'organismi',
 'di',
 'governo',
 'e',
 'di',
 'assemblee',
 'nazionali',
 'con']

In [275]:
prefixes = ['di','dal','nan','del','sug','nan','dei']
data.descrizione_completa = data.descrizione_completa.apply(lambda x: [i for i in x if not i.startswith(tuple(prefixes))])
data.descrizione_completa.iloc[0][:10]

['Membri',
 'organismi',
 'governo',
 'e',
 'assemblee',
 'nazionali',
 'con',
 'potestà',
 'legislativa',
 'e']

In [276]:
data.descrizione_completa = data.descrizione_completa.apply(lambda x: ' '.join(x))
data.descrizione_completa.iloc[0]

'Membri organismi governo e assemblee nazionali con potestà legislativa e regolamentare Membro consiglio ministri Membro senato Membro camera deputatinannandeputato deputata ministro ministra senatore senatricedeputata Parlamento europeo membro Parlamento europeo membro parlamento deputato deputata deputato Parlamento europeo membro camera deputati ministra governo ministro ministra ministro governo senatrice senatore'

In [277]:
train_set = data.iloc[:500,:]
test_set = data.iloc[500:,:]

In [296]:
len(data.Target.unique())

800

In [278]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_set.descrizione_completa)


In [279]:
X_train_counts

<500x10450 sparse matrix of type '<class 'numpy.int64'>'
	with 53711 stored elements in Compressed Sparse Row format>

In [280]:
# transform a count matrix to a normalized tf-idf representation (tf-idf transformer)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [281]:
X_train_tfidf

<500x10450 sparse matrix of type '<class 'numpy.float64'>'
	with 53711 stored elements in Compressed Sparse Row format>

In [286]:
knn = KNeighborsClassifier(n_neighbors=10)
# training our classifier ; train_data.target will be having numbers assigned for each category in train data
clf = knn.fit(X_train_tfidf, train_set.Target)


In [287]:
clf

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [288]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', knn),
])
# Fitting our train data to the pipeline
text_clf.fit(train_set.descrizione_completa, train_set.Target)

# Test data 
docs_test = test_set.descrizione_completa
# Predicting our test data
predicted = text_clf.predict(docs_test)
print('We got an accuracy of',np.mean(predicted == test_set.Target)*100, '% over the test data.')

We got an accuracy of 0.0 % over the test data.


In [290]:
test_set.Target

500                         Muratori in pietra e mattoni
501                              Casseronisti/cassonisti
502                 Muratori e formatori in calcestruzzo
503                        Carpentieri e falegnami edili
504                                        Ponteggiatori
                             ...                        
795    Manovali ed altro personale non qualificato de...
796    Manovali e personale non qualificato dell’edil...
797    Manovali e personale non qualificato della cos...
798    Personale non qualificato delle attività indus...
799                         Ufficiali delle forze armate
Name: Target, Length: 300, dtype: object

In [289]:
text_clf.predict(docs_test)

array(['Agronomi e forestali', 'Agronomi e forestali',
       'Agronomi e forestali', 'Agronomi e forestali',
       "Addetti all'immissione dati", 'Agronomi e forestali',
       'Brillatori e artificieri in cave e miniere',
       'Agenti del corpo forestale', 'Agenti del corpo forestale',
       'Agronomi e forestali',
       'Addetti alla preparazione e alla cottura di cibi in imprese per la ristorazione collettiva',
       'Addetti alla preparazione e alla cottura di cibi in imprese per la ristorazione collettiva',
       'Addetti alla preparazione e alla cottura di cibi in imprese per la ristorazione collettiva',
       'Addetti alla preparazione e alla cottura di cibi in imprese per la ristorazione collettiva',
       'Addetti alla preparazione e alla cottura di cibi in imprese per la ristorazione collettiva',
       'Addetti alla preparazione e alla cottura di cibi in imprese per la ristorazione collettiva',
       'Disegnatori tecnici',
       "Addetti all'informazione e all'as